In [1]:
import numpy as np
import torch
import config

from turb_vae.train2d import VAETrainer
torch.set_grad_enabled(False)


model = VAETrainer.load_from_checkpoint("../checkpoints/vae_kl_0p01.ckpt")
encoder = model.vae.encoder
data = config.TurbVaeConfig.test_dataset
N1 = 1000
dl = torch.utils.data.DataLoader(data, batch_size=N1, num_workers=15)

/home/hudson/code/spack/opt/spack/linux-ubuntu24.04-skylake_avx512/gcc-13.2.0/miniconda3-24.3.0-ree6kqamgcbjhoziwoelsymk7uzvnxju/envs/turb/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 56 worker processes in total. Our suggested max number of worker in current system is 20, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/hudson/code/spack/opt/spack/linux-ubuntu24.04-skylake_avx512/gcc-13.2.0/miniconda3-24.3.0-ree6kqamgcbjhoziwoelsymk7uzvnxju/envs/turb/lib/python3.11/site-packages/pytorch_lightning/utilities/par

In [2]:
batch = next(iter(dl))

In [3]:
N2 = 1000
n, L0 = batch
mu_logvar = encoder(n.to('cuda'))
mu = mu_logvar[:, :encoder.out_channels].view(-1, encoder.out_channels * 8 * 8)
logvar = mu_logvar[:, encoder.out_channels:].view(-1, encoder.out_channels * 8 * 8)

eps = torch.randn(N2, 1, 1).to("cuda")

z = mu + torch.exp(logvar / 2) * eps

# combine N1 and N2 dimensions
z = z.view(N1*N2, -1)
z.shape

torch.Size([1000000, 1024])

In [4]:
# mean stats
mu_est = z.mean(axis=0, keepdim=True)
mu_est.mean(), mu_est.std() / np.sqrt(N1*N2)

(tensor(-0.0124, device='cuda:0'), tensor(2.7543e-05, device='cuda:0'))

In [5]:
mu_est[:,:5].cpu()

tensor([[ 0.0047, -0.0055, -0.0086,  0.0557,  0.0122]])

In [6]:
# covariance stats
cov_est = torch.cov(z.T)
cov_est[:5, :5]

tensor([[1.1781, 0.4989, 0.2614, 0.1909, 0.2306],
        [0.4989, 1.2409, 0.5117, 0.2455, 0.2059],
        [0.2614, 0.5117, 1.0721, 0.3911, 0.2269],
        [0.1909, 0.2455, 0.3911, 0.9965, 0.3845],
        [0.2306, 0.2059, 0.2269, 0.3845, 0.9996]], device='cuda:0')

In [12]:
mu[0,:5], torch.exp(logvar[0,:5]/2)

(tensor([-1.8000, -2.4287, -1.6982, -0.7330, -0.5873], device='cuda:0'),
 tensor([0.0456, 0.0427, 0.0455, 0.0503, 0.0493], device='cuda:0'))